In [ ]:
import sys

In [ ]:
sys.path.insert(0,"../python/")
import analyzer_utils as au

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from bokeh.io import output_notebook
output_notebook()

# Get collections

In [ ]:
fina_eng_coll, db = au.get_coll("fina_bp_eng")
fina_hun_coll, _ = au.get_coll("fina_bp_hun")
bp_coll, _ = au.get_coll("budapest")

In [ ]:
colls = {
    "fina_bp_eng":fina_eng_coll,
    "fina_bp_hun":fina_hun_coll,
    "budapest":bp_coll
}

#### Number of documents in the collections

In [ ]:
for col in colls:
    size = db.command("collstats", col)["count"]
    print("Number of docs in %s: %i" % (col,size))

# Sample event

In [ ]:
doc = fina_eng_coll.find_one()

doc

# Upper and lower bound of the available datasets

In [ ]:
for col in colls:
    print()
    print("### %s ###" % col)
    print()
    print("Oldest events:")
    au.find_some_docs(colls[col],sort_params=[("id",1)],limit=3)
    print()
    print("Latest events:")
    au.find_some_docs(colls[col],limit=3)